<a href="https://colab.research.google.com/github/MichaelPaulukonis/notebooks/blob/main/fooocus_gdrive_colab_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fooocus - Run from Google Drive

**Google Colab Notebook**

Run [Fooocus Stable Diffusion](https://github.com/lllyasviel/Fooocus) without re-installing each time you load. Mount Google Drive and then run the code from there inside Colab!

Inspired by [fast-stable-diffusion](https://github.com/TheLastBen/fast-stable-diffusion)

GDrive-connected version by [Kitson Broadhurst](https://github.com/lllyasviel/Fooocus/issues/423#issuecomment-1863161412) - ([YouTube tutorial](https://youtu.be/EIWZ6u0o4dc))

In [1]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

#@markdown ---

Folder_Name = "focus_me" #@param {type:"string"}
#@markdown - Default folder name is `fooocus_sd`

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive
else:
  mainpth="MyDrive"

if Folder_Name!="":
  folderpath=Folder_Name
else:
  folderpath="fooocus_sd"

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [2]:
#@markdown # Install/Update fooocus repo
from IPython.utils import capture
from IPython.display import clear_output
import os
import time

if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using temporary colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth
  Shared_Drive=""

if Shared_Drive!="" and not os.path.exists("/content/gdrive/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  %mkdir -p /content/gdrive/$mainpth/$folderpath
  %cd /content/gdrive/$mainpth/$folderpath
  !git clone -q --branch main https://github.com/lllyasviel/Fooocus.git

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/$folderpath/Fooocus
  !git reset --hard
  !git checkout main
  time.sleep(1)
  !rm webui.sh
  !git pull
clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [3]:
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
import gdown
from gdown.download import get_url_from_gdrive_confirmation
import six
import re
from tqdm import tqdm

#@markdown # Model Download/Load

#@markdown Models are downloaded or moved to the `Fooocus/models/checkpoints/` folder. If they already exist in that folder you don't need to reload them.

#@markdown ---
PATH_to_MODEL = "" #@param {type:"string"}
#@markdown - Insert the full Google Drive path of your custom model

#@markdown OR
MODEL_LINK = "" #@param {type:"string"}
#@markdown - Insert the URL of the model you'd like to download


def getsrc(url):
    parsed_url = urlparse(url)
    if parsed_url.netloc == 'civitai.com':
        src='civitai'
    elif parsed_url.netloc == 'drive.google.com':
        src='gdrive'
    elif parsed_url.netloc == 'huggingface.co':
        src='huggingface'
    else:
        src='others'
    return src

src=getsrc(MODEL_LINK)

def get_name(url, gdrive):
    if not gdrive:
        response = requests.get(url, allow_redirects=False)
        if "Location" in response.headers:
            redirected_url = response.headers["Location"]
            quer = parse_qs(urlparse(redirected_url).query)
            if "response-content-disposition" in quer:
                disp_val = quer["response-content-disposition"][0].split(";")
                for vals in disp_val:
                    if vals.strip().startswith("filename="):
                        filenm=unquote(vals.split("=", 1)[1].strip())
                        return filenm.replace("\"","")
    else:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"}
        lnk="https://drive.google.com/uc?id={id}&export=download".format(id=url[url.find("/d/")+3:url.find("/view")])
        res = requests.session().get(lnk, headers=headers, stream=True, verify=True)
        res = requests.session().get(get_url_from_gdrive_confirmation(res.text), headers=headers, stream=True, verify=True)
        content_disposition = six.moves.urllib_parse.unquote(res.headers["Content-Disposition"])
        filenm = re.search(r"filename\*=UTF-8''(.*)", content_disposition).groups()[0].replace(os.path.sep, "_")
        return filenm


def dwn(url, dst, msg):
    file_size = None
    req = Request(url, headers={"User-Agent": "torch.hub"})
    u = urlopen(req)
    meta = u.info()
    if hasattr(meta, 'getheaders'):
        content_length = meta.getheaders("Content-Length")
    else:
        content_length = meta.get_all("Content-Length")
    if content_length is not None and len(content_length) > 0:
        file_size = int(content_length[0])

    with tqdm(total=file_size, disable=False, mininterval=0.5,
              bar_format=msg+' |{bar:20}| {percentage:3.0f}%') as pbar:
        with open(dst, "wb") as f:
            while True:
                buffer = u.read(8192)
                if len(buffer) == 0:
                    break
                f.write(buffer)
                pbar.update(len(buffer))
            f.close()


if (PATH_to_MODEL !=''):
  if os.path.exists(str(PATH_to_MODEL)):
    inf('\u2714 Using the trained model.','success', '200px')

  else:
      while not os.path.exists(str(PATH_to_MODEL)):
        inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
        PATH_to_MODEL=input()
      if os.path.exists(str(PATH_to_MODEL)):
        inf('\u2714 Using the custom model.','success', '200px')

  model=PATH_to_MODEL

elif MODEL_LINK != "":

      if src=='civitai':
         modelname=get_name(MODEL_LINK, False)
         model=f'/content/gdrive/{mainpth}/{folderpath}/Fooocus/models/checkpoints/{modelname}'
         if not os.path.exists(model):
            dwn(MODEL_LINK, model, 'Downloading the custom model')
            clear_output()
         else:
            inf('\u2714 Model already exists','primary', '300px')
      elif src=='gdrive':
         modelname=get_name(MODEL_LINK, True)
         model=f'/content/gdrive/{mainpth}/{folderpath}/Fooocus/models/checkpoints/{modelname}'
         if not os.path.exists(model):
            gdown.download(url=MODEL_LINK, output=model, quiet=False, fuzzy=True)
            clear_output()
         else:
            inf('\u2714 Model already exists','primary', '300px')
      else:
         modelname=os.path.basename(MODEL_LINK)
         model=f'/content/gdrive/{mainpth}/{folderpath}/Fooocus/models/checkpoints/{modelname}'
         if not os.path.exists(model):
            gdown.download(url=MODEL_LINK, output=model, quiet=False, fuzzy=True)
            clear_output()
         else:
            inf('\u2714 Model already exists','primary', '700px')

      if os.path.exists(model) and os.path.getsize(model) > 1810671599:
        inf('\u2714 Model downloaded, using the custom model.','success', '300px')
      else:
        !rm model
        inf('\u2718 Wrong link, check that the link is valid','danger', "300px")

#@markdown ---

In [ ]:
#@markdown # Run Fooocus

#@markdown Run this block last to install the required dependencies and start the app.

!pip install pygit2==1.15.1
%cd /content/gdrive/$mainpth/$folderpath/Fooocus
!python entry_with_update.py --share

/content/gdrive/MyDrive/focus_me/Fooocus
Already up-to-date
Update succeeded.
[System ARGV] ['entry_with_update.py', '--share']
Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Fooocus version: 2.5.5
[Cleanup] Attempting to delete content of temp dir /tmp/fooocus
[Cleanup] Cleanup successful
Total VRAM 15102 MB, total RAM 52217 MB
Set vram state to: NORMAL_VRAM
Always offload VRAM
Device: cuda:0 Tesla T4 : native
VAE dtype: torch.float32
Using pytorch cross attention
Refiner unloaded.
Running on local URL:  http://127.0.0.1:7865
IMPORTANT: You are using gradio version 3.41.2, however version 5.0.1 is available, please upgrade.
--------
Running on public URL: https://8b7261584af7a95bbf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
model_type EPS
UNet ADM Dimension 2816
Using pytorch attention in VAE
Working with z of shape (1, 4, 32, 32) = 4096 